# RAG - How to query

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

WEAVIATE_URL = os.getenv("WEAVIATE_URL")
WEAVIATE_KEY = os.getenv("WEAVIATE_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_URL = os.getenv("OPENAI_URL")

print(WEAVIATE_URL[:10])
print(WEAVIATE_KEY[:10])
print(OPENAI_API_KEY[:20])
print(OPENAI_URL)

http://loc
root-user-
sk-proj-iuwKF1Q94jnW
https://api.openai.com


## Connect to Weaviate

In [2]:
import weaviate
from weaviate.classes.init import Auth

# Connect to the local instance
client = weaviate.connect_to_local(
  host="127.0.0.1", # the address to the learner's instance
  port=8080,
  grpc_port=50051,
  auth_credentials=Auth.api_key(WEAVIATE_KEY),
  headers={
    "X-OpenAI-Api-Key": OPENAI_API_KEY
  }
)

print(client.is_ready())

True


### Start with (R) - Retrieval

In [3]:
wiki = client.collections.get("Wiki")

response = wiki.query.near_text(
    query="How do planes fly",
    limit=5,
    return_properties=["text", "title"]
)

for item in response.objects:
    print(item.properties)

{'text': 'A spin-off movie, Planes, was released on August 9, 2013. Its sequel Planes: Fire & Rescue was released on July 18, 2014. The Planes movies were made by DisneyToon Studios and not by Pixar. A third film was planned, but was cancelled due to the closure of DisneyToon Studios in June 2018.', 'title': 'Cars (movie)'}
{'text': "When flying under IFR, pilots fly set routes only, following airways and tracks between physical radio beacons and waypoints. While the pilot is still responsible to avoid incident, navigation under IFR is heavily reliant on an Air Traffic Controller's instructions.", 'title': 'Air navigation'}
{'text': 'The Stratocruiser had four piston engines. It had a pressurized cabin and two decks. Airlines could fly for much longer with the Stratocruiser, to places like Hawaii.', 'title': 'Boeing 377'}
{'text': 'Air will rise if it is heated by the ground or if it is deflected by mountains. Improvements in aerodynamics, in instruments and in the understanding of the

### Add (AG) - augmented generation - to make full RAG

#### Single Prompt

> Generate a response per **retrieved** object.

In [4]:
# Let's add some colour to our lives :)
BLUE   = "\033[94m"
PURPLE = "\033[95m"
RESET  = "\033[0"

In [7]:
from weaviate.classes.generate import GenerativeConfig

wiki = client.collections.get("Wiki")

response = wiki.generate.near_text(
    query="How do planes fly",
    auto_limit=1,
    limit=5,
    generative_provider=GenerativeConfig.openai(
        model="gpt-4o-mini",
        base_url=OPENAI_URL
    ),
    single_prompt="Please describe this to me {text}"

    # TODO: add GenerativeConfig with OpenAI and "gpt-4-1106-preview"
    # generative_provider=GenerativeConfig.

    # TODO: add a single prompt "Explain what this is about? {text}"
    # single_prompt=
)

# NOTE: the generated responses are included with the each object

for item in response.objects:
    print(f"{BLUE}=== Source ===")
    print(item.properties)

    print(f"{PURPLE}=== Generated Response ===")
    print(item.generative.text)
    # TODO: print the generative.text object
    # print(item.)

    print("\n")

=== Source ===
{'text': 'A spin-off movie, Planes, was released on August 9, 2013. Its sequel Planes: Fire & Rescue was released on July 18, 2014. The Planes movies were made by DisneyToon Studios and not by Pixar. A third film was planned, but was cancelled due to the closure of DisneyToon Studios in June 2018.', 'title': 'Cars (movie)', 'wiki_id': '20231101.simple_117715_3', 'url': 'https://simple.wikipedia.org/wiki/Cars%20%28movie%29'}
=== Generated Response ===
"Planes" is a spin-off movie from Disney's popular "Cars" franchise, released on August 9, 2013. Unlike the main "Cars" films, which were developed by Pixar, "Planes" was produced by DisneyToon Studios. The film follows Dusty Crophopper, a crop-dusting aircraft who dreams of becoming a racing champion. 

The sequel, "Planes: Fire & Rescue," debuted on July 18, 2014, and focuses on Dusty's adventures as he transitions from racing to becoming a firefighter, highlighting themes of bravery and teamwork.

Although a third install

#### Grouped Task

> Generate one response based on all **retrieved** objects.

In [8]:
wiki = client.collections.get("Wiki")

response = wiki.generate.near_text(
    query="How do planes fly",
    # auto_limit=1,
    limit=5,
    
    generative_provider=GenerativeConfig.openai(
        model="gpt-4-1106-preview",
        base_url=OPENAI_URL,
    ),
    grouped_task=" Explain the aerodynamics of flight, please only use provided context"

    # TODO: add a grouped task "Explain, how do planes fly? Please only use the provided content."
    # grouped_task=
)

print(f"{PURPLE}=== Generated Response ===")
print(response.generative.text)
# NOTE: group task response is at response.generative.text
# TODO: print the generated text
# print(response.)

print(f"{BLUE}=== Source ===")
for item in response.objects:
    print(item.properties)

=== Generated Response ===
The aerodynamics of flight involve the principles by which aircraft are able to fly through the air. While the provided context doesn't directly explain the fundamental principles of aerodynamics, we can infer some aspects based on the context of aircraft and air navigation.

Airplanes achieve flight through the four forces of aerodynamics: lift, weight (or gravity), thrust, and drag. Here’s a simple explanation of these principles based on the provided context:

1. Lift: Aircraft wings are designed to manipulate the air passing over and under them. This can be partly inferred from the context about the Boeing 717, which, like all aircraft, has wings that are crucial for flight. The shape of an airplane's wing (airfoil) causes air to flow faster over the top of the wing and slower beneath it, creating a difference in pressure that results in upward lift, as described in the context of "Gliding," where rising air currents help gliders stay aloft without engine

#### Specify which properties to use for grouped task

In [9]:
response = wiki.generate.near_text(
    query="How do planes fly",
    auto_limit=1,

    generative_provider=GenerativeConfig.openai(
        model="gpt-4-1106-preview",
        base_url=OPENAI_URL,
    ),

    grouped_task="Explain, how do planes fly? Please only use the provided content.",
    # TODO: add grouped properties to only use "text" and "title" 
    grouped_properties=["text", "title"]
)

print("=== Generated Response ===")
print(response.generative.text)

print("=== Source ===")
for item in response.objects:
    print(item.properties)

=== Generated Response ===
I'm sorry, but the provided content doesn't contain the technical or scientific information necessary to explain how planes fly. The content provided pertains to the "Cars" movie and its spin-off "Planes," as well as a brief mention about air navigation under Instrument Flight Rules (IFR). To explain how planes fly, you would need information on aerodynamics, aircraft design, the principles of lift, thrust, drag, and weight, and how these forces work together to enable an aircraft to achieve and sustain flight. Since that information is not included in the provided content, it's not possible to give you an accurate explanation based on the provided text.
=== Source ===
{'text': 'A spin-off movie, Planes, was released on August 9, 2013. Its sequel Planes: Fire & Rescue was released on July 18, 2014. The Planes movies were made by DisneyToon Studios and not by Pixar. A third film was planned, but was cancelled due to the closure of DisneyToon Studios in June 20

## Set default Generative model

In [10]:
from weaviate.classes.config import Reconfigure

wiki = client.collections.get("Wiki")

wiki.config.update(
    generative_config=Reconfigure.Generative.openai(
        model="gpt-4o-mini",
        base_url=OPENAI_URL
    )
    #TODO: set generative model
    # generative_config=Reconfigure.Generative.
)

> Try generative query without providing the model

In [11]:
response = wiki.generate.near_text(
    query="What african animals do we have info on. Please only list those provided in here.",
    auto_limit=1,
    
    grouped_task="Explain, how do planes fly? Please only use the provided content.",
)

print(f"{PURPLE}=== Generated Response ===")
print(response.generative.text)

print(f"{BLUE}=== Source ===")
for item in response.objects:
    print(item.properties)

=== Generated Response ===
The provided content does not contain any information about how planes fly. Therefore, I am unable to explain the principles of flight using the provided content. If you have other sources or materials related to aviation, feel free to share, and I can help explain how planes fly based on that information.
=== Source ===
{'text': 'Kigelia abyssinica, Kigelia acutifolia, Kigelia angolensis, Kigelia aethiopica, Kigelia aethiopum, Kigelia dinklagei, Kigelia elliotii, Kigelia elliptica, Kigelia erytraeae, Kigelia ikbaliae, Kigelia impressa, Kigelia lanceolata, Kigelia madagascariensis, Kigelia moosa, Kigelia perrottetti, Kigelia pinnata, Kigelia somalensis, Kigelia spragueana, Kigelia talbotii, Kigelia tristis', 'title': 'Kigelia', 'wiki_id': '20231101.simple_825692_1', 'url': 'https://simple.wikipedia.org/wiki/Kigelia'}
{'text': 'Taokhe Tati Tati River Taukhe Taung Taupse Taupye Tautswe Thamalakane River Thune Tshakujuwo Tsiklane Tuni River Tutumi River', 'title

## Close the client

In [12]:
client.close()